# Código para delimitar a quantidade de registros utilizados

## Versão do Python
O código foi executado na verão Python 3.12.8 (tags/v3.12.8:2dc476b, Dec  3 2024, 19:30:04) [MSC v.1942 64 bit (AMD64)] on win32
Type "help", "copyright", "credits" or "license" for more information. A exceção é o arquivo 3-Fine-tuning, pois o mesmo precisou ser processado no Colab e usou a versão 3.10 do Python.

## Quantidade registros
Conforme dito no vídeo de apresentação do Tech Challenge a base é muito grande para ser processada em computadores "normais".
Não estou dizendo que não dá para processar, mas o tempo necessário pode prejudicar a entrega do resultado esperado.
Por sugestão do professor, inicialmente limitei a 200 mil registros.
## Intervalo usado para quantidade de palavras
Defini a quantidade de palavras entre 10 e 200 para melhorar o tempo de treinamento e inferência, além de evitarmos assim, com uma boa margem de segurança, passar a quantidade máxima tokens por entrada para o modelo BERT (512 tokens), o GTP aceita um pouco mais de tokens. Ainda não tenho certeza do modelo que irei utilizar, então optei por ser conservador e usar o que aceita menos tokens para evitar ter que processar novamente os dados de entrada.

## Arquivos de dados e código fonte
Para facilitar o entendimento, os arquivos de código foram numerados. Dessa maneira a ordem de execução dos arquivos segue a numeração.
Os arquivos de dados devem estar na mesma pasta dos arquivos de código fonte.

Obs.: 
1 - Todas as linhas foram comentadas para facilitar o entendimento do código.
     
2 - Em função de uma prática que utilizo há algum tempo, todo o código é gerado na língua Inglesa. Tal hábito foi adquirido para facilitar os debates ou dúvidas sobre o meus código com pessoas de outras nacionalidades, como o código é para o Tech Challenge coloquei todos os comentários e mensagens em Português do Brasil.

3 - Indiretamente a base foi higienizada ao tratar o intervalo da quantidade de palavras e não aceitar dados nulos na amostra de 200.000 registros.

In [9]:
# Essa célula só precisa ser executada se as bibliotecas utilizadas na próxima célula não estiverem instaladas.
# !pip install gzip
# !pip install json
# !pip install random

In [1]:
import gzip  # Manipula arquivos .gz para leitura direta.
import json  # Trabalha com dados no formato JSON.
import random  # Embaralha os registros para diversidade nos dados de saída.
import os  # Garante o uso do diretório correto, independente de onde o script é executado.

In [11]:
print(os.getcwd())  # Mostra o diretório atual de execução

D:\PosTec\Fase 3\TechCallenge


In [12]:
# Verifica se o código está em um notebook ou script
try:
    script_dir = os.path.dirname(__file__)  # Diretório do script.
except NameError:
    script_dir = os.getcwd()  # Diretório atual (para notebooks).

# Caminho dos arquivos de entrada e saída
file_name = "trn.json.gz"
file_path = os.path.join(script_dir, file_name)  # Caminho completo para o arquivo compactado.
output_file = os.path.join(script_dir, "trn_filtered.json")  # Caminho completo para o arquivo de saída.

In [13]:
# Parâmetros de filtragem
MAX_SAMPLES = 200_000  # Limite máximo de registros.
MIN_CONTENT_LENGTH = 10  # Comprimento mínimo do conteúdo (em palavras).
MAX_CONTENT_LENGTH = 200  # Comprimento máximo do conteúdo (em palavras).

In [14]:
# Lista para armazenar os dados filtrados
filtered_data = []

# Abre o arquivo .gz e processa linha a linha
with gzip.open(file_path, 'rt', encoding='utf-8') as file:
    for line in file:  # Itera sobre cada linha do arquivo compactado
        record = json.loads(line)  # Converte a linha para um dicionário Python.
        content = record.get("content", "").strip()  # Obtém o campo 'content'.
        title = record.get("title", "").strip()  # Obtém o campo 'title'.

        # Filtragem: verifica se o 'content' está dentro do intervalo permitido de palavras
        if content and MIN_CONTENT_LENGTH <= len(content.split()) <= MAX_CONTENT_LENGTH:
            filtered_data.append({
                "input": f"Qual é a descrição de {title}?",  # Cria o campo 'input' com o título.
                "output": content  # O 'content' será usado como 'output'.
            })

        # Interrompe se atingir o limite de amostras
        if len(filtered_data) >= MAX_SAMPLES:
            break

In [15]:
# Embaralha os dados para diversidade
random.shuffle(filtered_data)

In [16]:
# Salva os dados filtrados no arquivo JSON
with open(output_file, "w", encoding="utf-8") as out_file:
    # Salva como uma lista JSON bem formatada
    json.dump(filtered_data, out_file, indent=2, ensure_ascii=False) # Indentação para facilitar leitura

In [17]:
# Imprime uma mensagem indicando onde os dados foram salvos e quantos registros foram incluídos
print(f"Dados filtrados salvos em {output_file}. Total de registros: {len(filtered_data)}")

Dados filtrados salvos em trn_filtered.json. Total de registros: 200000


In [19]:
with open("trn_filtered.json", "r", encoding="utf-8") as f:
    data = json.load(f)  # Tenta carregar o JSON, faz um teste para verificar se é possível abrir o arquivo.
    print(f"O arquivo JSON contém {len(data)} registros e é válido!")


O arquivo JSON contém 200000 registros e é válido!
